In [1]:
%matplotlib inline
from ztfquery import query
import numpy as np
import datetime
from tqdm import tqdm


In [18]:
class MultiNightSummary(query._ZTFTableHandler_):
    
    def __init__(self, start_date=None, end_date=None):
        self.nights = self.find_nights(start_date, end_date)
        
        print("Using {0} Nightly Sumaries between {1} and {2}".format(
            len(self.nights), self.nights[0], self.nights[-1]))
        
        self.data, self.missing_nights = self.stack_nights()
        
        print("Of these, {0} nights are missing because ZTF did not observe.".format(len(self.missing_nights)))
        
    @staticmethod
    def find_nights(start_date=None, end_date=None):
        date_format = "%Y%m%d"
    
        if start_date is None:
            now = datetime.datetime.now()
            start_time = now - datetime.timedelta(days=30)
        else:
            start_time = datetime.datetime.strptime(str(start_date), date_format)
        if end_date is None:
            end_time = datetime.datetime.now()
        else:
            end_time = datetime.datetime.strptime(str(end_date), date_format)
            
        if start_time > end_time:
            raise ValueError("Start time {0} occurs after end time {1}.".format(start_time, end_time))

        delta_t = (end_time - start_time).days
        
        dates = [(start_time + datetime.timedelta(days=x)).strftime(date_format) for x in range(0, delta_t + 1)]
        
        return dates
        
    def stack_nights(self):
        ns = None
        missing_nights = []
    
        for night in tqdm(self.nights):
            new_ns = self.get_ztf_data(night)
            
            if ns is None:
                ns = new_ns

            if new_ns is not None:
                ns.data = ns.data.append(new_ns.data)
            else:
                missing_nights.append(night)
                
        return ns.data, missing_nights
    
    @staticmethod
    def get_ztf_data(date=None):
        """Function to grab data for a given date using ztfquery. 
        Date should be given in format YYYYMMDD, with the day being the UT day for the END of the night.
        By default, today is selected. Returns a NightSummary if one is available, or None otherwise 
        (None is returned if there are no ZTF observations).
        """
        if date is None:
            print("No date specified. Assuming today.")
            now = datetime.datetime.now()
            date = now.strftime("%Y%m%d")
        try:
            return query.NightSummary(date)
        # query returns an index error is no ztf data is found
        except IndexError:
            return None
        
    @staticmethod
    def wrap_around_180(ra):
        ra[ra > np.pi] -= 2 * np.pi
        return ra
        
    def count_obs(self, ra_deg, dec_deg):
        ra_deg = np.degrees(self.wrap_around_180(np.radians(np.array([ra_deg]))))
        ras = np.degrees(self.wrap_around_180(np.array([np.radians(float(x)) for x in self.data["ra"]])))
#         ra_deg = np.degrees(ra)
#         dec_deg = np.degrees(dec)
        ztf_rad = 3.5 / np.cos(np.radians(dec_deg))

        obs = []      

        for i, x in enumerate(self.data["dec"]):            
            if np.logical_and(not dec_deg < float(x) - ztf_rad, not dec_deg > float(x) + ztf_rad):
                if np.logical_and(not ra_deg < float(ras[i]) - ztf_rad, not ra_deg > float(ras[i]) + ztf_rad):
                    obs.append(np.array(self.data)[i])

        return np.array(obs)

In [19]:
mns = MultiNightSummary(start_date=20190829)

  0%|          | 0/2 [00:00<?, ?it/s]

Using 2 Nightly Sumaries between 20190829 and 20190830


100%|██████████| 2/2 [00:02<00:00,  1.03s/it]

Of these, 0 nights are missing because ZTF did not observe.


In [20]:
mns.count_obs(44.540, -8.968)

array([['2019-08-29T10:32:28.4', '-32', '30', '1', 'targ', '351', '1',
        '47.4959', '-9.8500', '27.91', '14.3',
        'ztf_20190829439167_000351_zg', 'Kulkarni', 'all_sky'],
       ['2019-08-29T10:32:28.4', '-32', '30', '1', 'targ', '351', '1',
        '47.4959', '-9.8500', '27.91', '14.3',
        'ztf_20190829439167_000351_zg', 'Kulkarni', 'all_sky']],
      dtype=object)

In [ ]:
mns.show_gri_fields()